---

# **Importing of Libraries**

- In this section, we have imported all the libraries that we will used to scrap myntra products.

---

In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

---

# **Myntra Scraper**

- In this section, we will scrap the items as highlighted in the red box in the following image.

![img](./images/myntra-webpage.PNG?raw=true)

- We will use the following user defined function to extract products from myntra website.

---

In [4]:
def get_url(search_item, page):
    """Search the item using the url."""

    init_url = "https://www.myntra.com/{}"
    search_item = search_item.replace(" ", "-")
    final_url = init_url.format(search_item)
    final_url = final_url + "?p={}"
    return final_url.format(page)

def extractRow(item):
    """Extract details of each row of Myntra Product."""

    try:
        URL = 'https://www.myntra.com/' + item.find('a', {'target': '_blank'})['href']
    except AttributeError:
        URL = ''

    try:
        brand = item.find('h3', {'class': 'product-brand'}).text
    except AttributeError:
        brand = ''

    try:
        product_type = item.find('h4', {'class': 'product-product'}).text
    except AttributeError:
        product_type = ''

    try:
        sizes = item.find('h4', {'class': 'product-sizes'}).text
    except AttributeError:
        sizes = ''

    try:
        product_price = item.find('div', {'class': 'product-price'})

        try:
            discounted_price = product_price.find('span', {'class': 'product-discountedPrice'}).text
        except AttributeError:
            discounted_price = ''

        try:
            striked_price = product_price.find('span', {'class': 'product-strike'}).text
        except AttributeError:
            striked_price = ''

        try:
            discount_percent = product_price.find('span', {'class': 'product-discountPercentage'}).text
        except AttributeError:
            discount_percent = ''

        if (discounted_price == '') & (striked_price == '') & (discount_percent == ''):
            product_price = product_price.text
        else:
            product_price = ''

    except AttributeError:
        product_price = ''

    return (product_type, brand, sizes, product_price, discounted_price, striked_price, discount_percent, URL)

records = list()

def main(search, pages):
    """Executes the main program."""

    # Initialize the Chrome driver
    driver = webdriver.Chrome("chromedriver")

    record = list()

    for page in range(1, pages+1):
        driver.get(get_url(search, page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.findAll('li', {'class': 'product-base'})

        for result in results:
            record = extractRow(result)

            if record:
                records.append(record)

    driver.close()

    # # Saving file to a CSV

    cols = ['ProductType', 'Brand', 'Sizes', 'ProductPrice', 'DiscountedPrice', 'StrikedPrice', 'DiscountPercent', 'URL']
    data = pd.DataFrame(data=records, columns=cols)
    data.to_csv('results.csv', index=False)
    return data.head()

In [5]:
# Search and get the data associated with the product
search = input("Enter the Product:")
main(search=search, pages=2)

,ProductType,Brand,Sizes,ProductPrice,DiscountedPrice,StrikedPrice,DiscountPercent,URL
0,Sneakers,Puma,"Sizes: UK6, UK7, UK8, UK9, UK10, UK11",,Rs. 1399,Rs. 3499,(60% OFF),https://www.myntra.com/casual-shoes/puma/puma-...
1,Men Solid Loafers,Provogue,"Sizes: UK6, UK7, UK8, UK9, UK10, UK11",,Rs. 849,Rs. 3995,(Rs. 3146 OFF),https://www.myntra.com/casual-shoes/provogue/p...
2,Men Walking Shoes,Newfeel By Decathlon,"Sizes: UK5.5, UK6.5, UK7, UK8, UK8.5, UK9.5, U...",Rs. 899,,,,https://www.myntra.com/sports-shoes/newfeel-by...
3,Men Loafers,Fentacia,"Sizes: UK6, UK7, UK8, UK9, UK10, UK11, UK12",,Rs. 839,Rs. 3999,(79% OFF),https://www.myntra.com/casual-shoes/fentacia/f...
4,Men Trekking Shoes,Eego Italy,"Sizes: UK6, UK7, UK8, UK9, UK10, UK11",,Rs. 899,Rs. 2599,(Rs. 1700 OFF),https://www.myntra.com/sports-shoes/eego-italy...
